In [1]:
#KGAT_746a8c48819a19cbaf8ca0048244831b

In [2]:
!pip install opendatasets
!pip install pandas

Скачиваем

In [3]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/syahrulapriansyah2/myanimelist-2025")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: angrytea
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/syahrulapriansyah2/myanimelist-2025


100%|██████████| 9.57M/9.57M [00:00<00:00, 2.33GB/s]

Библиотеки

In [13]:
import opendatasets as od
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin
from sklearn.ensemble import GradientBoostingClassifier

Открываем датасет  
Удаляем строки, не имеющие в Members значения  
Задаем условие успешности: тайтл успешный, если у него 75% и больше зрителей: 1 - аниме успешное (собрало много просмотров) 0 - не успешное  
Для признаков возьмем Source, Type, Released_Season, Episodes, Genres, Studios, Demographic  
Пропуски по эпизодам заменяем медианой, по категориальным признакам на Unknown, жанрам и студиям поставим пустые строки  
Создаем список жанров  
Ищем самые популярные жанры и делаем для них колонки  
Список студий делаем  
Ищем популярные студии, не учитывая 'add some'  
Задаем колонки  
Собираем матрицу X из фич и назначаем целевую переменную успешности y  
Делим данные на train и test  
Числовые и бинарные признаки будем масштабировать, а для категориальных используем one-hot encoding  

In [5]:
pd.set_option("display.max_columns", None)
df = pd.read_csv("myanimelist-2025/mal_anime.csv")
df["Members"] = pd.to_numeric(df["Members"], errors="coerce")
df = df.dropna(subset=["Members"])
th = df["Members"].quantile(0.75)

df["hit"] = (df["Members"] >= th).astype(int)
df["Episodes"] = pd.to_numeric(df["Episodes"], errors="coerce")
m_episodes = df["Episodes"].median()
df["Episodes"] = df["Episodes"].fillna(m_episodes)

for col in ["Source", "Type", "Released_Season", "Demographic"]:
    df[col] = df[col].fillna("Unknown")

for col in ["Genres", "Studios"]: df[col] = df[col].fillna("")

df["Genres_l"] = df["Genres"].apply(lambda x: [i.strip() for i in str(x).split(",") if i.strip() != ""])
genre_count = Counter()

for g in df["Genres_l"]:
    genre_count.update(g)

top_g = [i for i, _ in genre_count.most_common(10)]

for g in top_g:
    col_name = f"genre_{g}"
    df[col_name] = df["Genres_l"].apply(lambda lst: int(g in lst))

g_cols = [f"genre_{g}" for g in top_g]
df["Studios_l"] = df["Studios"].apply(lambda x: [i.strip() for i in str(x).split(",") if i.strip() != ""])

studio_count = Counter()
for s in df["Studios_l"]:
  for st in s:
    clean_s = st.strip()
    if "add some" in clean_s.lower(): continue
    studio_count[clean_s] += 1

top_s = [i for i, _ in studio_count.most_common(10)]
for s in top_s:
    col_name = f"studio_{s}"
    df[col_name] = df["Studios_l"].apply(lambda lst: int(s in lst))

studio_cols = [f"studio_{s}" for s in top_s]

num = ["Episodes"] + g_cols + studio_cols
categ = ["Source", "Type", "Released_Season", "Demographic"]
feature = num + categ

X = df[feature]
y = df["hit"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)
pred = ColumnTransformer(transformers=[
    ("num", StandardScaler(),num),
    ("categ", OneHotEncoder(handle_unknown="ignore"), categ)
])

Бейзлайн

Точность получается 0.76, но F1-мера по целевому классу «успешное аниме» заметно ниже (0.24), что говорит о слабом улавливании именно редкого положительного класса. ROC-AUC около 0.73 показывает, что модель в среднем неплохо ранжирует объекты по вероятности успеха, но в точке принятия решения (0.5) серьёзно перекошена в сторону предсказаний класса 0.

In [6]:
gb_base = Pipeline(steps=[
    ("pred", pred),
    ("clf", GradientBoostingClassifier(
        random_state=42
    ))
])

gb_base.fit(X_train, y_train)

y_pred_gb = gb_base.predict(X_test)
y_proba_gb = gb_base.predict_proba(X_test)[:, 1]

print("Безйлайн")
print("Accuracy:", accuracy_score(y_test, y_pred_gb))
print("F1-score:", f1_score(y_test, y_pred_gb))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_gb))
print(classification_report(y_test, y_pred_gb))

Безйлайн
Accuracy: 0.762655601659751
F1-score: 0.2393617021276596
ROC-AUC: 0.7352991866698936
              precision    recall  f1-score   support

           0       0.77      0.97      0.86       903
           1       0.61      0.15      0.24       302

    accuracy                           0.76      1205
   macro avg       0.69      0.56      0.55      1205
weighted avg       0.73      0.76      0.70      1205



1 гипотеза - качество можно улучшить, если подобрать количество базовых деревьев, learning_rate и глубину деревьев в бустинге. Здесь запускаю GridSearchCV по нескольким разумным значениям и оптимизирую F1 по классу hit

На тесте точность остаётся примерно на уровне бейзлайна (0.76), но F1 растёт до 0.37 за счёт заметного увеличения recall и более сбалансированного качества между классами. ROC-AUC немного снижается относительно базовой модели (около 0.71 против 0.73), то есть общее качество ранжирования чуть хуже, но при выбранном пороге классификации модель лучше выделяет успешные тайтлы. Гипотеза подтверждается: за счёт настройки глубины и числа деревьев удалось ощутимо поднять F1 по целевому классу

In [7]:
param_grid_gb1 = {
    "clf__n_estimators": [100, 200, 300],
    "clf__learning_rate": [0.05, 0.1, 0.2],
    "clf__max_depth": [2, 3, 4]
}

gb_clf1 = Pipeline(steps=[
    ("pred", pred),
    ("clf", GradientBoostingClassifier(
        random_state=42
    ))
])

gb_gs1 = GridSearchCV(
    estimator=gb_clf1,
    param_grid=param_grid_gb1,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

gb_gs1.fit(X_train, y_train)

print("Лучшие параметры:", gb_gs1.best_params_)
print("Лучший F1 на CV:", gb_gs1.best_score_)

best_gb1 = gb_gs1.best_estimator_
y_pred_gb1 = best_gb1.predict(X_test)
y_proba_gb1 = best_gb1.predict_proba(X_test)[:, 1]

print("\nГипотеза 1")
print("Accuracy:", accuracy_score(y_test, y_pred_gb1))
print("F1-score:", f1_score(y_test, y_pred_gb1))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_gb1))
print(classification_report(y_test, y_pred_gb1))


Лучшие параметры: {'clf__learning_rate': 0.2, 'clf__max_depth': 4, 'clf__n_estimators': 300}
Лучший F1 на CV: 0.3658074559865983

Гипотеза 1
Accuracy: 0.7601659751037344
F1-score: 0.3676148796498906
ROC-AUC: 0.7113888216614229
              precision    recall  f1-score   support

           0       0.79      0.92      0.85       903
           1       0.54      0.28      0.37       302

    accuracy                           0.76      1205
   macro avg       0.67      0.60      0.61      1205
weighted avg       0.73      0.76      0.73      1205



2 гипотеза - число эпизодов сильно варьируется и может вести себя как длинный хвост, поэтому логарифмирование Episodes (log(1 + Episodes)) сделает распределение более сглаженным и может помочь бустингу чуть лучше разделять классы

Получившиеся лучшие параметры практически совпадают с гипотезой 1, а итоговые метрики на тесте (Accuracy, F1, ROC-AUC) фактически идентичны: лог-преобразование числа эпизодов не даёт дополнительного выигрыша по сравнению с уже настроенной моделью. Это значит, что для градиентного бустинга информация о числе эпизодов уже достаточно хорошо «отрабатывается» деревьями, и простое логарифмирование не даёт заметного эффекта. Гипотеза по сути не подтверждается — повышение качества по сравнению с гипотезой 1 отсутствует

In [8]:
df2 = df.copy()
df2["log_Episodes"] = np.log1p(df2["Episodes"])

num2 = ["log_Episodes"] + g_cols + studio_cols
categ2 = ["Source", "Type", "Released_Season", "Demographic"]
feature2 = num2 + categ2

X2 = df2[feature2]
y2 = df2["hit"]

X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2,
    test_size=0.2,
    random_state=42,
    stratify=y2
)

pred2 = ColumnTransformer(transformers=[
    ("num2", StandardScaler(), num2),
    ("categ", OneHotEncoder(handle_unknown="ignore"), categ2),
])

gb_clf2 = Pipeline(steps=[
    ("pred", pred2),
    ("clf", GradientBoostingClassifier(
        random_state=42
    ))
])

gb_gs2 = GridSearchCV(
    estimator=gb_clf2,
    param_grid=param_grid_gb1,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

gb_gs2.fit(X2_train, y2_train)

print("Лучшие параметры:", gb_gs2.best_params_)
print("Лучший F1 на CV:", gb_gs2.best_score_)

best_gb2 = gb_gs2.best_estimator_
y_pred_gb2 = best_gb2.predict(X2_test)
y_proba_gb2 = best_gb2.predict_proba(X2_test)[:, 1]

print("\nGradientBoosting + log(Episodes)")
print("Accuracy:", accuracy_score(y2_test, y_pred_gb2))
print("F1-score:", f1_score(y2_test, y_pred_gb2))
print("ROC-AUC:", roc_auc_score(y2_test, y_proba_gb2))
print(classification_report(y2_test, y_pred_gb2))


Лучшие параметры: {'clf__learning_rate': 0.2, 'clf__max_depth': 4, 'clf__n_estimators': 300}
Лучший F1 на CV: 0.3647641591686536

GradientBoosting + log(Episodes)
Accuracy: 0.7601659751037344
F1-score: 0.3676148796498906
ROC-AUC: 0.710589425975226
              precision    recall  f1-score   support

           0       0.79      0.92      0.85       903
           1       0.54      0.28      0.37       302

    accuracy                           0.76      1205
   macro avg       0.67      0.60      0.61      1205
weighted avg       0.73      0.76      0.73      1205



3 гипотеза - расширить топ до 20

На тесте точность немного снижается (0.75 против 0.76 в гипотезе 1), F1-мера (0.34) также чуть хуже, чем у лучшей модели из гипотезы 1, но всё ещё заметно выше исходного бейзлайна. ROC-AUC опускается до 0.706, что тоже хуже как базовой настройки, так и модели из гипотезы 1. В итоге расширение признакового пространства до top-20 жанров/студий не приносит дополнительного выигрыша — модель становится чуть сложнее, но по качеству слегка уступает лучшему варианту из гипотезы 1

In [9]:
genre_count_full = Counter()
for g_list in df["Genres_l"]:
    genre_count_full.update(g_list)

top_g20 = [i for i, _ in genre_count_full.most_common(20)]

for g in top_g20:
    col_name = f"genre20_{g}"
    df[col_name] = df["Genres_l"].apply(lambda lst: int(g in lst))

g_cols20 = [f"genre20_{g}" for g in top_g20]

studio_count_full = Counter()
for s_list in df["Studios_l"]:
    for st in s_list:
        clean_s = st.strip()
        if "add some" in clean_s.lower():
            continue
        studio_count_full[clean_s] += 1

top_s20 = [i for i, _ in studio_count_full.most_common(20)]

for s in top_s20:
    col_name = f"studio20_{s}"
    df[col_name] = df["Studios_l"].apply(lambda lst: int(s in lst))

studio_cols20 = [f"studio20_{s}" for s in top_s20]

num3 = ["Episodes"] + g_cols20 + studio_cols20
categ3 = ["Source", "Type", "Released_Season", "Demographic"]
feature3 = num3 + categ3

X3 = df[feature3]
y3 = df["hit"]

X3_train, X3_test, y3_train, y3_test = train_test_split(
    X3, y3,
    test_size=0.2,
    random_state=42,
    stratify=y3
)

pred3 = ColumnTransformer(transformers=[
    ("num3", StandardScaler(), num3),
    ("categ", OneHotEncoder(handle_unknown="ignore"), categ3),
])

gb_clf3 = Pipeline(steps=[
    ("pred", pred3),
    ("clf", GradientBoostingClassifier(
        random_state=42
    ))
])

gb_gs3 = GridSearchCV(
    estimator=gb_clf3,
    param_grid=param_grid_gb1,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

gb_gs3.fit(X3_train, y3_train)

print("Лучшие параметры:", gb_gs3.best_params_)
print("Лучший F1 на CV:", gb_gs3.best_score_)

best_gb3 = gb_gs3.best_estimator_
y_pred_gb3 = best_gb3.predict(X3_test)
y_proba_gb3 = best_gb3.predict_proba(X3_test)[:, 1]

print("\nGradientBoosting + top-20 жанров/студий")
print("Accuracy:", accuracy_score(y3_test, y_pred_gb3))
print("F1-score:", f1_score(y3_test, y_pred_gb3))
print("ROC-AUC:", roc_auc_score(y3_test, y_proba_gb3))
print(classification_report(y3_test, y_pred_gb3))


Лучшие параметры: {'clf__learning_rate': 0.2, 'clf__max_depth': 4, 'clf__n_estimators': 300}
Лучший F1 на CV: 0.3812519318027274

GradientBoosting + top-20 жанров/студий
Accuracy: 0.7477178423236515
F1-score: 0.341991341991342
ROC-AUC: 0.7056775428483422
              precision    recall  f1-score   support

           0       0.79      0.91      0.84       903
           1       0.49      0.26      0.34       302

    accuracy                           0.75      1205
   macro avg       0.64      0.59      0.59      1205
weighted avg       0.71      0.75      0.72      1205



Собственная реализация

Сначала описываю MyDecisionTreeRegressor - простое регрессионное дерево, которое на каждом шаге перебирает признаки и пороги, выбирает разбиение, минимизирующее MSE, и в листьях хранит среднее значение целевой переменной. Это дерево я использую как базовый алгоритм в бустинге.
Далее реализую MyGradientBoostingClassifier  
1) перевожу таргет в 0/1
2) начинаю с константного предсказания (средняя вероятность класса 1)
3) на каждой итерации считаю остатки между правильными метками и текущими предсказаниями, обучаю новое регрессионное дерево на этих остатках и добавляю его вклад к скору с шагом learning_rate
4) для предсказаний по новым объектам последовательно суммирую вклад всех деревьев и обрезаю результат в диапазон [0, 1], после чего трактую его как вероятность класса 1 и порогом 0.5 перевожу в метки классов


In [14]:
class MyDecisionTreeRegressor(BaseEstimator, RegressorMixin):
    def __init__(
        self,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=None
    ):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        n_samples, n_features = X.shape

        rng = np.random.RandomState(self.random_state)
        self.n_features_ = n_features
        self.tree_ = []

        def mse(values):
            if values.size == 0:
                return 0.0
            mean = values.mean()
            return np.mean((values - mean) ** 2)

        def best_split(X_node, y_node, depth):
            n_samples_node, n_features_node = X_node.shape
            if n_samples_node < self.min_samples_split:
                return None, None, None, None
            if self.max_depth is not None and depth >= self.max_depth:
                return None, None, None, None

            best_feature = None
            best_threshold = None
            best_score = np.inf
            best_left_idx = None
            best_right_idx = None

            for feature in range(n_features_node):
                values = X_node[:, feature]
                thresholds = np.unique(values)
                for thr in thresholds:
                    left_idx = values <= thr
                    right_idx = values > thr
                    if left_idx.sum() < self.min_samples_leaf or right_idx.sum() < self.min_samples_leaf:
                        continue
                    mse_left = mse(y_node[left_idx])
                    mse_right = mse(y_node[right_idx])
                    score = (left_idx.sum() * mse_left + right_idx.sum() * mse_right) / n_samples_node
                    if score < best_score:
                        best_score = score
                        best_feature = feature
                        best_threshold = thr
                        best_left_idx = left_idx
                        best_right_idx = right_idx

            if best_feature is None:
                return None, None, None, None
            return best_feature, best_threshold, best_left_idx, best_right_idx

        def build_tree(X_node, y_node, depth):
            node = {}
            node["value"] = float(y_node.mean())

            feature, threshold, left_idx, right_idx = best_split(X_node, y_node, depth)
            if feature is None:
                node["feature"] = None
                node["threshold"] = None
                node["left"] = None
                node["right"] = None
                return node

            node["feature"] = feature
            node["threshold"] = threshold
            node["left"] = build_tree(X_node[left_idx], y_node[left_idx], depth + 1)
            node["right"] = build_tree(X_node[right_idx], y_node[right_idx], depth + 1)
            return node

        self.tree_ = build_tree(X, y, depth=0)
        return self

    def _predict_one(self, x, node):
        while node["feature"] is not None:
            if x[node["feature"]] <= node["threshold"]:
                node = node["left"]
            else:
                node = node["right"]
        return node["value"]

    def predict(self, X):
        X = np.asarray(X)
        n_samples = X.shape[0]
        y_pred = np.zeros(n_samples, dtype=float)
        for i in range(n_samples):
            y_pred[i] = self._predict_one(X[i], self.tree_)
        return y_pred

class MyGradientBoostingClassifier(BaseEstimator, ClassifierMixin):
    def __init__(
        self,
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=None
    ):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        self.classes_ = np.unique(y)
        if self.classes_.shape[0] != 2:
            raise ValueError("MyGradientBoostingClassifier поддерживает только бинарную классификацию")

        y_bin = (y == self.classes_[1]).astype(float)
        n_samples = X.shape[0]

        base_pred = np.full(n_samples, y_bin.mean())
        self.base_pred_ = base_pred.mean()

        self.trees_ = []

        F = np.full(n_samples, self.base_pred_, dtype=float)

        for m in range(self.n_estimators):
            residual = y_bin - F

            tree = MyDecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf
            )
            tree.fit(X, residual)
            self.trees_.append(tree)

            update = tree.predict(X)
            F += self.learning_rate * update

        return self

    def _raw_predict(self, X):
        X = np.asarray(X)
        F = np.full(X.shape[0], self.base_pred_, dtype=float)
        for tree in self.trees_:
            F += self.learning_rate * tree.predict(X)
        return F

    def predict_proba(self, X):
        F = self._raw_predict(X)
        p1 = np.clip(F, 1e-5, 1 - 1e-5)
        p0 = 1.0 - p1
        return np.vstack([p0, p1]).T

    def predict(self, X):
        proba = self.predict_proba(X)
        y_bin = (proba[:, 1] >= 0.5).astype(int)
        return self.classes_[y_bin]

Бйзлайн

In [15]:
my_gb_base = Pipeline(steps=[
    ("pred", pred),
    ("clf", MyGradientBoostingClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42
    ))
])

my_gb_base.fit(X_train, y_train)

y_pred_my_gb = my_gb_base.predict(X_test)
y_proba_my_gb = my_gb_base.predict_proba(X_test)[:, 1]

print("Бейзлайн")
print("Accuracy:", accuracy_score(y_test, y_pred_my_gb))
print("F1-score:", f1_score(y_test, y_pred_my_gb))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_my_gb))
print(classification_report(y_test, y_pred_my_gb))


Бейзлайн
Accuracy: 0.762655601659751
F1-score: 0.23118279569892472
ROC-AUC: 0.7336472244835098
              precision    recall  f1-score   support

           0       0.77      0.97      0.86       903
           1       0.61      0.14      0.23       302

    accuracy                           0.76      1205
   macro avg       0.69      0.56      0.55      1205
weighted avg       0.73      0.76      0.70      1205



1 гипотеза

In [16]:
param_grid_my_gb1 = {
    "clf__n_estimators": [100, 200, 300],
    "clf__learning_rate": [0.05, 0.1, 0.2],
    "clf__max_depth": [2, 3, 4]
}

my_gb_clf1 = Pipeline(steps=[
    ("pred", pred),
    ("clf", MyGradientBoostingClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42
    ))
])

my_gb_gs1 = GridSearchCV(
    estimator=my_gb_clf1,
    param_grid=param_grid_my_gb1,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

my_gb_gs1.fit(X_train, y_train)

print("Лучшие параметры:", my_gb_gs1.best_params_)
print("Лучший F1 на CV:", my_gb_gs1.best_score_)

best_my_gb1 = my_gb_gs1.best_estimator_
y_pred_my_gb1 = best_my_gb1.predict(X_test)
y_proba_my_gb1 = best_my_gb1.predict_proba(X_test)[:, 1]

print("\nГипотеза 1")
print("Accuracy:", accuracy_score(y_test, y_pred_my_gb1))
print("F1-score:", f1_score(y_test, y_pred_my_gb1))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_my_gb1))
print(classification_report(y_test, y_pred_my_gb1))


Лучшие параметры: {'clf__learning_rate': 0.2, 'clf__max_depth': 4, 'clf__n_estimators': 300}
Лучший F1 на CV: 0.36529977068965763

Гипотеза 1
Accuracy: 0.7634854771784232
F1-score: 0.3595505617977528
ROC-AUC: 0.7188712386232792
              precision    recall  f1-score   support

           0       0.79      0.93      0.85       903
           1       0.56      0.26      0.36       302

    accuracy                           0.76      1205
   macro avg       0.68      0.60      0.61      1205
weighted avg       0.73      0.76      0.73      1205



2 гипотеза

In [17]:
my_gb_clf2 = Pipeline(steps=[
    ("pred", pred2),
    ("clf", MyGradientBoostingClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42
    ))
])

my_gb_gs2 = GridSearchCV(
    estimator=my_gb_clf2,
    param_grid=param_grid_my_gb1,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

my_gb_gs2.fit(X2_train, y2_train)

print("Лучшие параметры:", my_gb_gs2.best_params_)
print("Лучший F1 на CV:", my_gb_gs2.best_score_)

best_my_gb2 = my_gb_gs2.best_estimator_
y_pred_my_gb2 = best_my_gb2.predict(X2_test)
y_proba_my_gb2 = best_my_gb2.predict_proba(X2_test)[:, 1]

print("\nMyGradientBoosting + log(Episodes)")
print("Accuracy:", accuracy_score(y2_test, y_pred_my_gb2))
print("F1-score:", f1_score(y2_test, y_pred_my_gb2))
print("ROC-AUC:", roc_auc_score(y2_test, y_proba_my_gb2))
print(classification_report(y2_test, y_pred_my_gb2))


Лучшие параметры: {'clf__learning_rate': 0.2, 'clf__max_depth': 4, 'clf__n_estimators': 300}
Лучший F1 на CV: 0.36529977068965763

MyGradientBoosting + log(Episodes)
Accuracy: 0.7634854771784232
F1-score: 0.3595505617977528
ROC-AUC: 0.7188712386232792
              precision    recall  f1-score   support

           0       0.79      0.93      0.85       903
           1       0.56      0.26      0.36       302

    accuracy                           0.76      1205
   macro avg       0.68      0.60      0.61      1205
weighted avg       0.73      0.76      0.73      1205



3 гипотеза

In [18]:
my_gb_clf3 = Pipeline(steps=[
    ("pred", pred3),
    ("clf", MyGradientBoostingClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42
    ))
])

my_gb_gs3 = GridSearchCV(
    estimator=my_gb_clf3,
    param_grid=param_grid_my_gb1,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

my_gb_gs3.fit(X3_train, y3_train)

print("Лучшие параметры:", my_gb_gs3.best_params_)
print("Лучший F1 на CV:", my_gb_gs3.best_score_)

best_my_gb3 = my_gb_gs3.best_estimator_
y_pred_my_gb3 = best_my_gb3.predict(X3_test)
y_proba_my_gb3 = best_my_gb3.predict_proba(X3_test)[:, 1]

print("\nMyGradientBoosting + top-20 жанров/студий ")
print("Accuracy:", accuracy_score(y3_test, y_pred_my_gb3))
print("F1-score:", f1_score(y3_test, y_pred_my_gb3))
print("ROC-AUC:", roc_auc_score(y3_test, y_proba_my_gb3))
print(classification_report(y3_test, y_pred_my_gb3))


Лучшие параметры: {'clf__learning_rate': 0.2, 'clf__max_depth': 4, 'clf__n_estimators': 300}
Лучший F1 на CV: 0.3773161705172974

MyGradientBoosting + top-20 жанров/студий 
Accuracy: 0.7560165975103734
F1-score: 0.3287671232876712
ROC-AUC: 0.7019849214905429
              precision    recall  f1-score   support

           0       0.78      0.93      0.85       903
           1       0.53      0.24      0.33       302

    accuracy                           0.76      1205
   macro avg       0.66      0.58      0.59      1205
weighted avg       0.72      0.76      0.72      1205

